<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
import requests
from bs4 import BeautifulSoup

In [174]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [175]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [176]:
# текст запроса
query_3_1 = f'''
    select distinct count(*)
    from public.vacancies
'''

In [177]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
print(df)

C:\Users\Иван\AppData\Local\Temp\ipykernel_12200\2173422648.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


   count
0  49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [178]:
# текст запроса
query_3_2 = '''
    select distinct count(*)
    from public.employers
'''

In [179]:
# результат запроса
df = pd.read_sql_query(query_3_2, connection)
print(df)

   count
0  23501


C:\Users\Иван\AppData\Local\Temp\ipykernel_12200\2813488741.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_2, connection)


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [180]:
# текст запроса
query_3_3 = f'''
    select distinct count(*)
    from public.areas
'''

In [181]:
# результат запроса
df = pd.read_sql_query(query_3_3, connection)
print(df)

C:\Users\Иван\AppData\Local\Temp\ipykernel_12200\1558151174.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_3, connection)


   count
0   1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [182]:
# текст запроса
query_3_4 = f'''
    select distinct count(*)
    from public.industries
'''

In [183]:
# результат запроса
df = pd.read_sql_query(query_3_4, connection)
print(df)

   count
0    294


C:\Users\Иван\AppData\Local\Temp\ipykernel_12200\729770387.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_4, connection)


***

### выводы по предварительному анализу данных  
В базе 49 тысяч вакансий, это довольно мало, учитывая что в базе представлены 1362 региона и 23500 работодателей.  
294 сферы деятельности => присутствует разнообразие областей, в которых можно поискать вакансии

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [184]:
# текст запроса
query_4_1 = f'''
    select
        ar.name,
        count(*)
    from public.vacancies vac
    join public.areas ar on ar.id = vac.area_id
    group by ar.name
    order by count(*) desc
'''

In [185]:
# результат запроса
df = pd.read_sql_query(query_4_1, connection)
print(df)

C:\Users\Иван\AppData\Local\Temp\ipykernel_12200\1574996576.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_1, connection)


                name  count
0             Москва   5333
1    Санкт-Петербург   2851
2              Минск   2112
3        Новосибирск   2006
4             Алматы   1892
..               ...    ...
764       Тарко-Сале      1
765    Новоаннинский      1
766            Бирск      1
767           Сасово      1
768             Анна      1

[769 rows x 2 columns]


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [186]:
# текст запроса
query_4_2 = f'''
    select count(*)
    from public.vacancies vac
    where (vac.salary_from is not null) or (vac.salary_to is not null)
'''

In [187]:
# результат запроса
df = pd.read_sql_query(query_4_2, connection)
print(df)

   count
0  24073


C:\Users\Иван\AppData\Local\Temp\ipykernel_12200\1991716859.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_2, connection)


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [188]:
# текст запроса
query_4_3 = f'''
    with from_ as (
        select avg(vac.salary_from)
        from public.vacancies vac
        where vac.salary_from is not null
    ), to_ as (
        select avg(vac.salary_to)
        from public.vacancies vac
        where vac.salary_to is not null
    )
    select * from from_
    union all select * from to_
'''

In [189]:
# результат запроса
df = pd.read_sql_query(query_4_3, connection)
print(df)

             avg
0   71064.657901
1  110536.741923


C:\Users\Иван\AppData\Local\Temp\ipykernel_12200\3795284598.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_3, connection)


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [190]:
# текст запроса
query_4_4 = f'''
    select
        schedule,
        employment,
        count(*)
    from public.vacancies
    group by schedule, employment
    order by count(*) desc
'''

In [191]:
# результат запроса
df = pd.read_sql_query(query_4_4, connection)
print(df)

C:\Users\Иван\AppData\Local\Temp\ipykernel_12200\512236719.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_4, connection)


            schedule           employment  count
0        Полный день     Полная занятость  35367
1   Удаленная работа     Полная занятость   7802
2      Гибкий график     Полная занятость   1593
3   Удаленная работа  Частичная занятость   1312
4     Сменный график     Полная занятость    940
5        Полный день           Стажировка    569
6     Вахтовый метод     Полная занятость    367
7        Полный день  Частичная занятость    347
8      Гибкий график  Частичная занятость    312
9        Полный день     Проектная работа    141
10  Удаленная работа     Проектная работа    133
11     Гибкий график           Стажировка    116
12    Сменный график  Частичная занятость    101
13  Удаленная работа           Стажировка     64
14     Гибкий график     Проектная работа     18
15    Сменный график           Стажировка     12
16    Вахтовый метод     Проектная работа      2
17    Сменный график     Проектная работа      1


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [192]:
# текст запроса
query_4_5 = f'''
    select
        experience,
        count(*)
    from public.vacancies
    group by experience
    order by count(*)
'''

In [193]:
# результат запроса
df = pd.read_sql_query(query_4_5, connection)
print(df)

           experience  count
0         Более 6 лет   1337
1           Нет опыта   7197
2       От 3 до 6 лет  14511
3  От 1 года до 3 лет  26152


C:\Users\Иван\AppData\Local\Temp\ipykernel_12200\3410932330.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_5, connection)


***

### Выводы по детальному анализу вакансий  
Лидеры по количеству вакансий - это весьма развитые крупные города миллионники, такие как Москва, Санкт-Петербург, Минск.  
Лишь у половины вакансий заполнено хотя бы одно поле с зарплатой. Это свидетельствует о том, что либо работодали небрежно заполняют объявления, либо заработную плату на многих вакансия трудно оценить или она сильно варьируется в зависимости от кандидата.  
Средняя нижняя граница з/п 71000, это средняя з/п Junior разработчиков. Средняя верхняя граница з/п 110000, что является довольно низким показателем, так как большинство высоко квалифицированных специалистов в IT получают больше 200000 руб. Верхний показатель такой низкий, потому что на 26 тыс. вакансий требуются люди с опытом от 1 до 3 лет(это в большинстве случаев Junior разработчики). С другой стороны это хорошо, так как среди соискателей довольно много начинающих специалистов.  
Большинство вакансий (около 70%) требуют полную занятость и полный рабочий день.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [194]:
# текст запроса
query_5_1 = f'''
    select
        emp.name,
        count(*)
    from public.vacancies vac
    join public.employers emp on emp.id = vac.employer_id
    group by emp.name
    order by count(*) desc
    limit 5
'''

In [195]:
# результат запроса
df = pd.read_sql_query(query_5_1, connection)
print(df)

C:\Users\Иван\AppData\Local\Temp\ipykernel_12200\1368165621.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_1, connection)


            name  count
0         Яндекс   1933
1     Ростелеком    491
2       Тинькофф    444
3           СБЕР    428
4  Газпром нефть    331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [225]:
# текст запроса
query_5_2_1 = f'''
    select
        ar.name,
        count(emp.name)
    from public.vacancies vac
    right join public.areas ar on ar.id = vac.area_id
    left join public.employers emp on ar.id = emp.area
    group by ar.name
    having count(vac.id) = 0
    order by 2 desc
'''

In [226]:
# результат запроса
df1 = pd.read_sql_query(query_5_2_1, connection)
print(df1)

C:\Users\Иван\AppData\Local\Temp\ipykernel_12200\3884540736.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1 = pd.read_sql_query(query_5_2_1, connection)


                             name  count
0                          Россия    410
1                       Казахстан    207
2              Московская область     75
3              Краснодарский край     19
4              Ростовская область     18
..                            ...    ...
588                      Струнино      0
589    Славгород (Алтайский край)      0
590                     Сорочинск      0
591  Ленинский (Тульская область)      0
592                         Тавда      0

[593 rows x 2 columns]


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [198]:
# текст запроса
query_5_3 = f'''
    select
        emp.name,
        count(distinct ar.name)
    from public.vacancies vac
    join public.areas ar on ar.id = vac.area_id
    join public.employers emp on vac.employer_id = emp.id
    group by emp.name
    order by 2 desc
'''

In [199]:
# результат запроса
df = pd.read_sql_query(query_5_3, connection)
print(df)

C:\Users\Иван\AppData\Local\Temp\ipykernel_12200\2424163555.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_3, connection)


                         name  count
0                      Яндекс    181
1                  Ростелеком    152
2                  Спецремонт    116
3      Поляков Денис Иванович     88
4                   ООО ЕФИН      71
...                       ...    ...
14761                  UniSol      1
14762            UNISTORY LLC      1
14763                   UNIT6      1
14764     United Distribution      1
14765              UNITEL LLC      1

[14766 rows x 2 columns]


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [200]:
# текст запроса
query_5_4 = f'''
    select
        emp.name
    from public.employers emp
    left join public.employers_industries emp_ind on emp.id = emp_ind.employer_id
    left join public.industries ind on ind.id = emp_ind.industry_id
    where ind.name is null
    order by 1
'''

In [201]:
# результат запроса
df = pd.read_sql_query(query_5_4, connection)
print(df)

C:\Users\Иван\AppData\Local\Temp\ipykernel_12200\2049463723.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_4, connection)


                                                   name
0     1000 мелочей (ИП Гафиатуллина Гульшат Митарисо...
1                                             1001 Soft
2                                             100 печей
3                                              101 Плюс
4                                           1221Системс
...                                                 ...
8414           Ярославский завод упаковочных материалов
8415                           Ярославский филиал ПГУПС
8416    Ярославский ЦГМС - филиал ФГБУ Центральное УГМС
8417                                       Ярпож Казань
8418                            Яшнова Алена Викторовна

[8419 rows x 1 columns]


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [202]:
# текст запроса
query_5_5 = f'''
    select
        emp.name,
        count(*)
    from public.employers_industries emp_ind
    join public.employers emp on emp.id = emp_ind.employer_id
    group by emp.name
    having count(*) = 4
    order by 1
'''

In [203]:
# результат запроса
df = pd.read_sql_query(query_5_5, connection)
print(df)

                 name  count
0        101 Интернет      4
1            21vek.by      4
2                2ГИС      4
3                  2К      4
4         4 пикселя +      4
...               ...    ...
1131            ЮРИОН      4
1132  ЮТИП Технологии      4
1133           ЯКласс      4
1134            ЯрНео      4
1135    Яшин&Партнёры      4

[1136 rows x 2 columns]


C:\Users\Иван\AppData\Local\Temp\ipykernel_12200\1574912611.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_5, connection)


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [204]:
# текст запроса
query_5_6 = f'''
    select
        count(*)
    from public.employers_industries emp_ind
    join public.employers emp on emp.id = emp_ind.employer_id
    join public.industries ind on ind.id = emp_ind.industry_id
    where ind.name = 'Разработка программного обеспечения'
'''

In [205]:
# результат запроса
df = pd.read_sql_query(query_5_6, connection)
print(df)

C:\Users\Иван\AppData\Local\Temp\ipykernel_12200\1176301150.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_6, connection)


   count
0   3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [206]:
# код для получения списка городов-милионников
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
table1 = page.find('table', class_ = 'standard sortable')

headers = []
for i in table1.find_all('th')[:3]:
    title = i.text
    headers.append(title)

mydata = pd.DataFrame(columns = headers)
for j in table1.find_all('tr')[1:]:
    row_data = j.find_all('td')
    row = [i.text for i in row_data][:3]
    length = len(mydata)
    mydata.loc[length] = row
print(mydata)

     №            Город Население,  тыс. чел.  (на 1 января 2023 года)
0    1           Москва                                         13 098
1    2  Санкт-Петербург                                          5 598
2    3      Новосибирск                                          1 635
3    4     Екатеринбург                                          1 539
4    5           Казань                                          1 315
5    6  Нижний Новгород                                          1 213
6    7       Красноярск                                          1 197
7    8        Челябинск                                          1 183
8    9           Самара                                          1 164
9   10              Уфа                                          1 158
10  11   Ростов-на-Дону                                          1 136
11  12        Краснодар                                          1 121
12  13             Омск                                          1 111
13  14

In [207]:
# текст запроса
query_5_7 = f'''
    select
        ar.name,
        count(*)
    from public.vacancies vac
    join public.areas ar on ar.id = vac.area_id
    join public.employers emp on emp.id = vac.employer_id
    where emp.name = 'Яндекс'
    group by ar.name
    order by 2 desc
'''

In [208]:
# результат запроса
df = pd.read_sql_query(query_5_7, connection)
# print(df)
yandex = mydata.merge(df, left_on = 'Город', right_on = 'name', how = 'left')
yandex = yandex.drop(labels = ['Население,  тыс. чел.  (на 1 января 2023 года)', '№', 'Город'], axis = 1)
yandex.loc[len(yandex)] = ['Total', yandex['count'].sum()]
print(yandex)

               name  count
0            Москва     54
1   Санкт-Петербург     42
2       Новосибирск     35
3      Екатеринбург     39
4            Казань     25
5   Нижний Новгород     36
6        Красноярск     23
7         Челябинск     22
8            Самара     26
9               Уфа     26
10   Ростов-на-Дону     25
11        Краснодар     30
12             Омск     21
13          Воронеж     32
14            Пермь     25
15        Волгоград     24
16            Total    485


C:\Users\Иван\AppData\Local\Temp\ipykernel_12200\3012760276.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_7, connection)


***

### выводы по анализу работодателей  
Большинство вакансий предлагают одни из самых крупных и развитых IT компаний России - это Яндекс и Ростелеком. Далее идут крупные банки Сбер и Тинькофф, в которых IT-специалистов требуется не сильно меньше.  
Прослеживается корреляция в размере компании и количестве регионов, в которых она публикует вакансии. Филиалы более крупных и развитых компаний можно найти в большем количестве регионов.  
У 35% работадателей не указана сфера дефтельности, что несколько затрудняет соискателям поиск подходящей вакансии, а аналитикам не позволяет выстроить полную картину состояния рынка трудоустройства для работников в сфере Data Science.  


# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [209]:
# текст запроса
query_6_1 = f'''
    select
        *
    from public.vacancies
    where (lower(name) like '%data%') or (lower(name) like '%данн%')
'''

In [210]:
# результат запроса
df = pd.read_sql_query(query_6_1, connection)
print(df.shape)

C:\Users\Иван\AppData\Local\Temp\ipykernel_12200\2401383304.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_1, connection)


(1771, 10)


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [211]:
# текст запроса
query_6_2 = f'''
    select
        *
    from public.vacancies
    where
        (
            (lower(name) like '%data scientist%') or
            (lower(name) like '%data science%') or 
            (lower(name) like '%исследователь данных%') or
            ((lower(name) like '%ml%') and (lower(name) not like '%html%')) or
            (lower(name) like '%machine learning%') or
            (lower(name) like '%машинн%обучен%')
        ) and
        (
            (lower(name) like '%junior%') or
            (lower(experience) like 'нет опыта') or
            (lower(employment) like 'стажировка')
        )
'''

In [212]:
# результат запроса
df = pd.read_sql_query(query_6_2, connection)
print(df)

C:\Users\Иван\AppData\Local\Temp\ipykernel_12200\2386638782.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_2, connection)


          id                                               name  \
0   50533416                            Junior+ CV/ML developer   
1   54825318                           Data Scientist (Junior+)   
2   55145641                              Junior Data Scientist   
3   55553897                     Junior Data Scientist (Стажер)   
4   55529384     Математик, Junior/Middle/Senior Data Scientist   
5   55343709  Data Scientist/Machine Learning Engineer (Deve...   
6   54475492                            Data Scientist (Junior)   
7   55119989                       Intern/Junior Data Scientist   
8   55120211                         Junior Data Scientist (VK)   
9   55120401                       Junior/Middle Data Scientist   
10  55390769          Data Scientist (Computer Vision) - Junior   
11  52158791  Исследователь в области машинного обучения и м...   
12  52396406                             Data Scientist, Junior   
13  55373640  Junior Data Scientist Управления модельных рис..

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [213]:
# текст запроса
query_6_3 = f'''
    with vacancies_ as (
        select *
        from public.vacancies
        where
            (lower(name) like '%data scientist%') or
            (lower(name) like '%data science%') or 
            (lower(name) like '%исследователь данных%') or
            ((name like '%ML%') and (name not like '%HTML%')) or
            (lower(name) like '%machine learning%') or
            (lower(name) like '%машинн%обучен%')
    )
    select *
    from vacancies_
    where (key_skills like '%SQL%') or (key_skills like '%postgres%')
'''

In [214]:
# результат запроса
df = pd.read_sql_query(query_6_3, connection)
print(df.shape)

C:\Users\Иван\AppData\Local\Temp\ipykernel_12200\2342775382.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_3, connection)


(201, 10)


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [215]:
# текст запроса
query_6_4 = f'''
    with vacancies_ as (
        select *
        from public.vacancies
        where
            (lower(name) like '%data scientist%') or
            (lower(name) like '%data science%') or 
            (lower(name) like '%исследователь данных%') or
            ((name like '%ML%') and (name not like '%HTML%')) or
            (lower(name) like '%machine learning%') or
            (lower(name) like '%машинн%обучен%')
    )
    select *
    from vacancies_
    where (key_skills like '%Python%')
'''

In [216]:
# результат запроса
df = pd.read_sql_query(query_6_4, connection)
print(df.shape)

C:\Users\Иван\AppData\Local\Temp\ipykernel_12200\361131444.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_4, connection)


(351, 10)


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [217]:
# текст запроса
query_6_5 = f'''
    with vacancies_ as (
        select *
        from public.vacancies
        where
            (lower(name) like '%data scientist%') or
            (lower(name) like '%data science%') or 
            (lower(name) like '%исследователь данных%') or
            ((name like '%ML%') and (name not like '%HTML%')) or
            (lower(name) like '%machine learning%') or
            (lower(name) like '%машинн%обучен%')
    )
    select
        key_skills,
        cardinality(string_to_array(key_skills, '\t')) as array
    from vacancies_ vac
    union all
    select
        'mean',
        avg(cardinality(string_to_array(key_skills, '\t')))
    from vacancies_ vac
    
'''

In [218]:
# результат запроса
df = pd.read_sql_query(query_6_5, connection)
print(df)

C:\Users\Иван\AppData\Local\Temp\ipykernel_12200\4051561159.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_5, connection)


                                            key_skills     array
0            Python\tSQL\tРабота в команде\tOpenCV\tML  5.000000
1    Oracle Pl/SQL\tБазы данных\tMachine Learning\t...  9.000000
2    Python\tOpenCV\tTensorflow\tComputer Vision\tН...  8.000000
3                       Python\tFlask\tLinux\tAWS\tGit  5.000000
4     Python\tSQL\tNLP\tScikit-learn\tMachine Learning  5.000000
..                                                 ...       ...
476           Python\tScikit-learn\tPandas\tGit\tNumpy  5.000000
477  Qt\tООП\tC++\tMS Visual Studio\tРазработка ком...  6.000000
478  SCALA\tSQL\tМатематический анализ\tСтатистичес...  7.000000
479  Python\tLinux\tKubernetes\tAWS\tGit\tPeople Ma...  6.000000
480                                               mean  6.406032

[481 rows x 2 columns]


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [219]:
# текст запроса
query_6_6 = f'''
    with vacancies_ as (
        select *
        from public.vacancies
        where
            (lower(name) like '%data scientist%') or
            (lower(name) like '%data science%') or 
            (lower(name) like '%исследователь данных%') or
            ((name like '%ML%') and (name not like '%HTML%')) or
            (lower(name) like '%machine learning%') or
            (lower(name) like '%машинн%обучен%')
    )
    select
        experience,
        avg((coalesce(salary_to, salary_from) + coalesce(salary_from, salary_to)) / 2)
    from vacancies_
    where (salary_from is not null) or (salary_to is not null)
    group by experience
'''

In [220]:
# результат запроса
df = pd.read_sql_query(query_6_6, connection)
print(df)

C:\Users\Иван\AppData\Local\Temp\ipykernel_12200\1300537.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_6, connection)


           experience            avg
0       От 3 до 6 лет  243114.666667
1  От 1 года до 3 лет  139674.750000
2           Нет опыта   74642.857143


***

## выводы по предметному анализу
В нашем датасете представлена 1771 вакансия, имеющая отношение к данным, что составляет всего 3.6% от общего числа IT-вакансий. Для начинающих специалистов этой области, представлено всего 51 вакансия. Из этого можно сделать вывод, что во времена создания датасета специалисты по данным не пользовались особой популярностью. Особенно мало вакансий для людей, не имеющих опыта работы в этой сфере.  
В 201 вакансии требуются знания SQL или Postgres, следовательно каждый специалист должен владеть этими навыками, так как SQL один из самых популярных языков запросов.  
351 вакансия требует знаний языка Python, и это неудивительно, потому что этот язык в последнее время набирает большую популярность во многих IT-сферах, благодаря своей простоте и универсальности.  
В среднем работадатели в требованиях указывают 6 навыков. Это говорит о том, что специалисты в этой сфере должны уметь работать с довольно большим количеством инструментов и технологий.  

# Общий вывод по проекту

Крупные и развитые IT-компании, такие как Яндекс, Сбер, Газпром и др. предосталвяют большего всего вакансий с более высокой средней заплатой. Большинство их вакансий находятся в крупных городах. Маленькие компании обычно предлагают от 1 до 5 вакансий, с меньшей средней з/п.  
Средняя верхняя граница зарплаты 110000 рублей, что довольно низкий показатель. Возможно из-за того, что представлено довольно много вакансий для начинающих специалистов (с опытом 1 - 3 года) или много вакансий в маленьких компаниях в маленьких городах(требует дополнительного исследования).  
Треть работадателей не указывают сферу деятельности, что не очень приятно при поиске работы и при анализе рынка труда.  
В целом довольно высокие требования для специалистов в сфере Data Science и довольно мало вакансий для этой профессии без опыта работы.
